In [ ]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 44.1 MB/s eta 0:00:00


In [ ]:
from pgmpy.models import DiscreteBayesianNetwork as DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [ ]:
model_structure = DiscreteBayesianNetwork([
    ('Burglary', 'Alarm'),
    ('Earthquake', 'Alarm'),
    ('Alarm', 'JohnCalls'),
    ('Alarm', 'MaryCalls')
])

In [ ]:
cpd_burglary = TabularCPD(variable='Burglary', variable_card=2, values=[[0.999], [0.001]])
# for 0: 0.999 and for 1: 0.001
cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2, values=[[.998], [0.002]])

In [ ]:
print(cpd_burglary)

+-------------+-------+
| Burglary(0) | 0.999 |
+-------------+-------+
| Burglary(1) | 0.001 |
+-------------+-------+


In [ ]:
print(cpd_earthquake)

+---------------+-------+
| Earthquake(0) | 0.998 |
+---------------+-------+
| Earthquake(1) | 0.002 |
+---------------+-------+


In [ ]:
cpd_alarm = TabularCPD(
    variable='Alarm',
    variable_card=2,
    evidence=['Burglary', 'Earthquake'],
    evidence_card=[2, 2],
    values=[[0.999, 0.71, 0.06, 0.05], [0.001, 0.29, 0.94, 0.95]]
)

In [ ]:
print(cpd_alarm)

+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.999         | 0.71          | 0.06          | 0.05          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.001         | 0.29          | 0.94          | 0.95          |
+------------+---------------+---------------+---------------+---------------+


In [ ]:
cpd_johncalls = TabularCPD(
    variable='JohnCalls',
    variable_card=2,
    evidence=['Alarm'],
    evidence_card=[2],
    values=[[0.95, 0.1], [0.05, 0.9]]
)
print(cpd_johncalls)

+--------------+----------+----------+
| Alarm        | Alarm(0) | Alarm(1) |
+--------------+----------+----------+
| JohnCalls(0) | 0.95     | 0.1      |
+--------------+----------+----------+
| JohnCalls(1) | 0.05     | 0.9      |
+--------------+----------+----------+


In [ ]:
cpd_marycalls = TabularCPD(
    variable='MaryCalls',
    variable_card=2,
    evidence=['Alarm'],
    evidence_card=[2],
    values=[[0.99, 0.3], [0.01, 0.7]]
)
print(cpd_marycalls)

+--------------+----------+----------+
| Alarm        | Alarm(0) | Alarm(1) |
+--------------+----------+----------+
| MaryCalls(0) | 0.99     | 0.3      |
+--------------+----------+----------+
| MaryCalls(1) | 0.01     | 0.7      |
+--------------+----------+----------+


In [ ]:
model_structure.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_johncalls, cpd_marycalls)

In [ ]:
model_structure.check_model()

True

In [ ]:
inference = VariableElimination(model_structure)

QUERY 1: P(JohnCall, MaryCall, Alarm | Burglary = 0, Earthquake = 0)

In [ ]:
result1 = inference.query(
    variables=['JohnCalls', 'MaryCalls', 'Alarm'],
    evidence={'Burglary': 0, 'Earthquake': 0}
)
print(result1)

+--------------+--------------+----------+----------------------------------+
| JohnCalls    | MaryCalls    | Alarm    |   phi(JohnCalls,MaryCalls,Alarm) |
+==============+==============+==========+==================================+
| JohnCalls(0) | MaryCalls(0) | Alarm(0) |                           0.9396 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(0) | MaryCalls(0) | Alarm(1) |                           0.0000 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(0) | MaryCalls(1) | Alarm(0) |                           0.0095 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(0) | MaryCalls(1) | Alarm(1) |                           0.0001 |
+--------------+--------------+----------+----------------------------------+
| JohnCalls(1) | MaryCalls(0) | Alarm(0) |                           0.0495 |
+--------------+--------------+----------+----------------------

In [ ]:
print("Probability that Alarm will ring when JohnCalls and MaryCalls is True: ", result1.values[1][1][1])

Probability that Alarm will ring when JohnCalls and MaryCalls is True:  0.0006300000000000001


QUERY 2: P(JohnCalls)

In [ ]:
result2 = inference.query(variables=['JohnCalls'])
print(result2)

+--------------+------------------+
| JohnCalls    |   phi(JohnCalls) |
+==============+==================+
| JohnCalls(0) |           0.9479 |
+--------------+------------------+
| JohnCalls(1) |           0.0521 |
+--------------+------------------+


In [ ]:
print("Probability that JohnCalls = True : ", result2.values[1])

Probability that JohnCalls = True :  0.0521389757


QUERY 3: P(Burglary | JohnCalls, MaryCalls)

In [ ]:
result3 = inference.query(
    variables=['Burglary'],
    evidence={'JohnCalls': 1, 'MaryCalls': 1}
)
print(result3)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+


In [ ]:
print("Probability that Burglary will happen when JohnCalls and MaryCalls is True: ", result3.values[1])

Probability that Burglary will happen when JohnCalls and MaryCalls is True:  0.284171835364393


QUERY 4: P(Alarm | Burglary = 1, Earthquake = 0)

In [ ]:
result4 = inference.query(
    variables=['Alarm'],
    evidence={'Burglary': 1, 'Earthquake': 0}
)
print(result4)

+----------+--------------+
| Alarm    |   phi(Alarm) |
+==========+==============+
| Alarm(0) |       0.0600 |
+----------+--------------+
| Alarm(1) |       0.9400 |
+----------+--------------+


In [ ]:
print("Probability that Alarm will ring when Burglary is True but Earthquake is False: ", result4.values[1])

Probability that Alarm will ring when Burglary is True but Earthquake is False:  0.94


QUERY 5: P(JohnCalls, MaryCalls|Alarm = 0)

In [ ]:
result5 = inference.query(
    variables=['JohnCalls', 'MaryCalls'],
    evidence={'Alarm': 0}
)
print(result5)

+--------------+--------------+----------------------------+
| JohnCalls    | MaryCalls    |   phi(JohnCalls,MaryCalls) |
+==============+==============+============================+
| JohnCalls(0) | MaryCalls(0) |                     0.9405 |
+--------------+--------------+----------------------------+
| JohnCalls(0) | MaryCalls(1) |                     0.0095 |
+--------------+--------------+----------------------------+
| JohnCalls(1) | MaryCalls(0) |                     0.0495 |
+--------------+--------------+----------------------------+
| JohnCalls(1) | MaryCalls(1) |                     0.0005 |
+--------------+--------------+----------------------------+


In [ ]:
print("Probability that JohnCalls and MaryCalls when Alarm didnt ring: ", result5.values[1][1])

Probability that JohnCalls and MaryCalls when Alarm didnt ring:  0.0005000000000000001
